In [7]:
import pandas as pd
import numpy as np

In [2]:
df= pd.read_csv('trajecdata.csv')

In [3]:
df

,TrackID,xmin,ymin,xmax,ymax,frame,lost,occuluded,generated,label
0,0,833,1634,857,1715,1501,0,0,1,Pedestrian
1,0,834,1634,858,1715,1502,0,0,1,Pedestrian
2,0,834,1638,858,1719,1503,0,0,1,Pedestrian
3,0,836,1638,860,1719,1504,0,0,1,Pedestrian
4,0,836,1638,862,1719,1505,0,0,1,Pedestrian
...,...,...,...,...,...,...,...,...,...,...
151665,126,751,81,783,154,2027,0,0,1,Pedestrian
151666,126,751,81,783,154,2028,0,0,1,Pedestrian
151667,126,751,81,783,154,2029,0,0,1,Pedestrian
151668,126,751,81,783,154,2030,0,0,1,Pedestrian


In [ ]:
#Pedestrain speed

In [4]:
# Compute the center points
df['x_center'] = (df['xmin'] + df['xmax']) / 2
df['y_center'] = (df['ymin'] + df['ymax']) / 2


In [ ]:
#calculate speed

In [23]:
# Calculate the center of bounding box for each frame
df['x_center'] = (df['xmin'] + df['xmax']) / 2
df['y_center'] = (df['ymin'] + df['ymax']) / 2

# Calculate displacement and speed
df = df.sort_values(by=['TrackID', 'frame'])
df['displacement'] = np.sqrt((df['x_center'].diff())**2 + (df['y_center'].diff())**2)
df.loc[df['TrackID'] != df['TrackID'].shift(), 'displacement'] = 0
df['speed'] = df['displacement'] # assuming 1 time-step interval

# Calculate Euclidean distance from destination
destination = df.groupby('TrackID').tail(1)[['TrackID', 'x_center', 'y_center']].set_index('TrackID')
df['dest_x'] = df['TrackID'].map(destination['x_center'])
df['dest_y'] = df['TrackID'].map(destination['y_center'])
df['dest_distance'] = np.sqrt((df['dest_x'] - df['x_center'])**2 + (df['dest_y'] - df['y_center'])**2)

# Euclidean distance from the nearest pedestrian was calculated as 'nearest_ped_dist'


In [24]:
df

,TrackID,xmin,ymin,xmax,ymax,frame,lost,occuluded,generated,label,x_center,y_center,x_speed,y_speed,speed,dist_from_dest,displacement,dest_x,dest_y,dest_distance
1,0,834,1634,858,1715,1502,0,0,1,Pedestrian,846.0,1674.5,25.0,0.0,0.000000,136.624485,0.000000,1137.0,1877.5,354.809808
2,0,834,1638,858,1719,1503,0,0,1,Pedestrian,846.0,1678.5,0.0,100.0,4.000000,132.959580,4.000000,1137.0,1877.5,352.536523
3,0,836,1638,860,1719,1504,0,0,1,Pedestrian,848.0,1678.5,50.0,0.0,2.000000,132.159941,2.000000,1137.0,1877.5,350.887446
4,0,836,1638,862,1719,1505,0,0,1,Pedestrian,849.0,1678.5,25.0,0.0,1.000000,131.769685,1.000000,1137.0,1877.5,350.064280
5,0,838,1642,862,1723,1506,0,0,1,Pedestrian,850.0,1682.5,25.0,100.0,4.123106,127.695928,4.123106,1137.0,1877.5,346.978385
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
151665,126,751,81,783,154,2027,0,0,1,Pedestrian,767.0,117.5,50.0,0.0,2.000000,1687.748574,2.000000,769.0,121.5,4.472136
151666,126,751,81,783,154,2028,0,0,1,Pedestrian,767.0,117.5,0.0,0.0,0.000000,1687.748574,0.000000,769.0,121.5,4.472136
151667,126,751,81,783,154,2029,0,0,1,Pedestrian,767.0,117.5,0.0,0.0,0.000000,1687.748574,0.000000,769.0,121.5,4.472136
151668,126,751,81,783,154,2030,0,0,1,Pedestrian,767.0,117.5,0.0,0.0,0.000000,1687.748574,0.000000,769.0,121.5,4.472136


In [25]:
import numpy as np

def calculate_nearest_ped_dist(group):
    x = group['x_center'].values
    y = group['y_center'].values
    coordinates = np.column_stack((x, y))
    if len(coordinates) == 1:  # If there's only one pedestrian, distance is infinity
        return [np.inf]
    else:
        distance_matrix = np.linalg.norm(coordinates - coordinates[:, None], axis=-1)
        np.fill_diagonal(distance_matrix, np.inf)  # We fill the diagonal with np.inf to avoid zero distance (to self)
        nearest_ped_dist = np.min(distance_matrix, axis=1)
        return nearest_ped_dist

df['nearest_ped_dist'] = df.groupby('frame').apply(calculate_nearest_ped_dist).explode().values


In [26]:
df

,TrackID,xmin,ymin,xmax,ymax,frame,lost,occuluded,generated,label,...,y_center,x_speed,y_speed,speed,dist_from_dest,displacement,dest_x,dest_y,dest_distance,nearest_ped_dist
1,0,834,1634,858,1715,1502,0,0,1,Pedestrian,...,1674.5,25.0,0.0,0.000000,136.624485,0.000000,1137.0,1877.5,354.809808,16.530275
2,0,834,1638,858,1719,1503,0,0,1,Pedestrian,...,1678.5,0.0,100.0,4.000000,132.959580,4.000000,1137.0,1877.5,352.536523,23.4094
3,0,836,1638,860,1719,1504,0,0,1,Pedestrian,...,1678.5,50.0,0.0,2.000000,132.159941,2.000000,1137.0,1877.5,350.887446,138.55775
4,0,836,1638,862,1719,1505,0,0,1,Pedestrian,...,1678.5,25.0,0.0,1.000000,131.769685,1.000000,1137.0,1877.5,350.064280,16.530275
5,0,838,1642,862,1723,1506,0,0,1,Pedestrian,...,1682.5,25.0,100.0,4.123106,127.695928,4.123106,1137.0,1877.5,346.978385,134.265595
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
151665,126,751,81,783,154,2027,0,0,1,Pedestrian,...,117.5,50.0,0.0,2.000000,1687.748574,2.000000,769.0,121.5,4.472136,40.660177
151666,126,751,81,783,154,2028,0,0,1,Pedestrian,...,117.5,0.0,0.0,0.000000,1687.748574,0.000000,769.0,121.5,4.472136,116.361506
151667,126,751,81,783,154,2029,0,0,1,Pedestrian,...,117.5,0.0,0.0,0.000000,1687.748574,0.000000,769.0,121.5,4.472136,27.018512
151668,126,751,81,783,154,2030,0,0,1,Pedestrian,...,117.5,0.0,0.0,0.000000,1687.748574,0.000000,769.0,121.5,4.472136,35.387145


In [ ]:
#LSTM model

In [27]:
input_seq_len = 8
output_seq_len = 12

def create_sequences(data, input_seq_len, output_seq_len):
    inputs = []
    outputs = []

    for i in range(len(data) - input_seq_len - output_seq_len):
        inputs.append(data.iloc[i : i + input_seq_len])
        outputs.append(data.iloc[i + input_seq_len : i + input_seq_len + output_seq_len])

    return np.array(inputs), np.array(outputs)

inputs, outputs = create_sequences(df[['x_center', 'y_center']], input_seq_len, output_seq_len)


In [28]:
from sklearn.model_selection import train_test_split

inputs_train, inputs_test, outputs_train, outputs_test = train_test_split(inputs, outputs, test_size=0.2, random_state=42)


In [30]:
from tensorflow.keras.layers import Reshape

model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(input_seq_len, 2)))
model.add(Dense(2 * output_seq_len))  # 2 for x and y
model.add(Reshape((output_seq_len, 2)))  # Reshaping the output to match the actual output shape
model.compile(optimizer='adam', loss='mse')

model.fit(inputs_train, outputs_train, epochs=10, verbose=0)

predictions = model.predict(inputs_test)

# Calculate ADE and FDE
ade = np.mean(np.sqrt(np.sum(np.square(outputs_test - predictions), axis=-1)))
fde = np.sqrt(np.sum(np.square(outputs_test[:, -1] - predictions[:, -1]), axis=-1))


948/948 [==============================] - 3s 3ms/step


In [31]:
ade 

15.769535003013486

In [33]:
predictions

array([[[ 318.74677 ,  995.0298  ],
        [ 320.38803 ,  996.7207  ],
        [ 320.27368 ,  996.2064  ],
        ...,
        [ 320.8288  ,  992.1431  ],
        [ 322.622   ,  993.3565  ],
        [ 321.7767  ,  994.4206  ]],

       [[  21.150837,  898.7659  ],
        [  22.48951 ,  896.15643 ],
        [  23.815142,  898.8696  ],
        ...,
        [  27.48059 ,  897.304   ],
        [  26.948278,  899.0613  ],
        [  27.89559 ,  897.7709  ]],

       [[1100.5359  , 1897.6299  ],
        [1102.8302  , 1902.1125  ],
        [1102.2595  , 1901.0448  ],
        ...,
        [1102.1843  , 1893.2314  ],
        [1105.0574  , 1895.243   ],
        [1102.7621  , 1896.0615  ]],

       ...,

       [[ 745.31024 ,  889.2216  ],
        [ 746.1274  ,  888.83264 ],
        [ 746.6461  ,  889.1589  ],
        ...,
        [ 744.7456  ,  888.05804 ],
        [ 745.3097  ,  888.7704  ],
        [ 744.3266  ,  889.50824 ]],

       [[ 797.7981  ,  947.0578  ],
        [ 798.6054  ,  946.

In [ ]:
#near collisions

In [36]:
preds=predictions

In [37]:
def calculate_near_collisions(preds, threshold=0.1):
    near_collisions = 0
    total_interactions = 0
    for frame in np.unique(preds['frame']):
        # Get all pedestrians in the current frame
        frame_data = preds[preds['frame'] == frame]
        positions = frame_data[['x_center', 'y_center']].values
        
        # Compute all pairwise distances
        pairwise_distances = np.sqrt(np.sum((positions[:, None] - positions)**2, axis=-1))
        
        # Count the interactions that are near-collisions
        near_collisions += np.sum(pairwise_distances < threshold) - len(frame_data)  # Subtract self-interactions
        
        total_interactions += len(frame_data)*(len(frame_data) - 1)  # Subtract self-interactions

    return (near_collisions / total_interactions) * 100

percentage_near_collisions = calculate_near_collisions(preds)
print("Percentage of Near-Collisions: ", percentage_near_collisions)


IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices